In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dateutil

In [101]:
# CPI Graph

df_CPI = pd.read_csv("CPIAUCSL.csv")
df_CPI['DATE'] = pd.to_datetime(df_CPI['DATE'], format='%Y-%m-%d')
df_CPI["CPI_change"] =  df_CPI["CPIAUCSL"].pct_change(12)*100
df_CPI = df_CPI[(df_CPI['DATE'] >= '2000-01-01') & (df_CPI['DATE'] <= '2022-06-01')]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_CPI['DATE'], y=df_CPI["CPI_change"], name="CPI"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="CPI Percentage Change (2000 to 2022)",
    font=dict(
        size=18,
    )
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Hawkish-Dovish Measure*10", secondary_y=False)
fig.update_yaxes(title_text="CPI Percentage Change", secondary_y=True)

fig.show()


In [102]:
#PPI Graph

df_PPI = pd.read_csv("PPIACO.csv")
df_PPI['DATE'] = pd.to_datetime(df_PPI['DATE'], format='%Y-%m-%d')
df_PPI["PPI_change"] =  df_PPI["PPIACO"].pct_change(12)*100
df_PPI = df_PPI[(df_PPI['DATE'] >= '2000-01-01') & (df_PPI['DATE'] <= '2022-06-01')]


fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_PPI['DATE'], y=df_PPI["PPI_change"], name="PPI"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_CPI['DATE'], y=df_CPI["CPI_change"], name="CPI"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="PPI & CPI Percentage Change (2000 to 2022)",
    font=dict(
        size=18,
    )
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="CPI Percentage Change", secondary_y=False)
fig.update_yaxes(title_text="PPI Percentage Change", secondary_y=True)

fig.show()

In [103]:
# Market Volatility Indicator

df_MVI = pd.read_csv("EMVMACROCONSUME.csv")
df_MVI['DATE'] = pd.to_datetime(df_MVI['DATE'], format='%Y-%m-%d')
# df_MVI["MVI_change"] =  df_MVI["EMVMACROCONSUME"]
# df_MVI["MVI_change"] =  df_MVI["EMVMACROCONSUME"].pct_change(12)*100
df_MVI = df_MVI[(df_MVI['DATE'] >= '2000-01-01') & (df_MVI['DATE'] <= '2022-06-01')]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_MVI['DATE'], y=df_MVI["EMVMACROCONSUME"], name="MVI"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Market Volatility Indicator (2000 to 2022)",
    font=dict(
        size=18,
    )
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Placeholder", secondary_y=False)
fig.update_yaxes(title_text="Market Volatility Index", secondary_y=True)

fig.show()

In [104]:
# Stability Results and PPI

df_MVI = pd.read_csv("EMVMACROCONSUME.csv")
df_MVI['DATE'] = pd.to_datetime(df_MVI['DATE'], format='%Y-%m-%d')
df_stability = pd.read_csv("stability_results_combined.csv")

df_stability['date'] = pd.to_datetime(df_stability['date'], format='%Y-%m-%d')
df_hawk = pd.read_csv("hawk_results_combined.csv")
print(df_hawk.head)
# df_hawk['date'] = pd.to_datetime(df_hawk['date'], format='%Y-%m-%d')
df_hawk['date'] = pd.to_datetime(df_hawk['date'])

df_MVI = df_MVI[(df_MVI['DATE'] >= '2000-01-01') & (df_MVI['DATE'] <= '2022-06-01')]
df_hawk = df_hawk[(df_hawk['date'] >= '2000-01-01') & (df_hawk['date'] <= '2022-06-01')]
df_stability = df_stability[(df_stability['date'] >= '2000-01-01') & (df_stability['date'] <= '2022-06-01')]
df_hawk = df_hawk.sort_values(by='date')
df_hawk['year_month'] = df_hawk['date'].dt.to_period('M')
df_hawk['year_month'] = df_hawk['year_month'].dt.to_timestamp()
df_hawk = df_hawk.sort_values(by='year_month')
df_stability = df_stability.sort_values(by='date')
df_stability['year_month'] = df_stability['date'].dt.to_period('M')
df_stability['year_month'] = df_stability['year_month'].dt.to_timestamp()
df_stability = df_stability.sort_values(by='year_month')

hawkish_df = df_hawk[df_hawk['label'] == 'HAWKISH']
dovish_df = df_hawk[df_hawk['label'] == 'DOVISH']

stable_df = df_stability[df_stability['label'] == 'STABLE']
volatile_df = df_stability[df_stability['label'] == 'VOLATILE']

stable_totals = stable_df.groupby('year_month')['score'].sum().reset_index(name='stable_total')
volatile_totals = stable_df.groupby('year_month')['score'].sum().reset_index(name='volatile_total')


hawkish_totals = hawkish_df.groupby('year_month')['score'].sum().reset_index(name='hawkish_total')
dovish_totals = dovish_df.groupby('year_month')['score'].sum().reset_index(name='dovish_total')

monthly_totals = hawkish_totals.merge(dovish_totals, how='outer', on='year_month')

monthly_totals = monthly_totals.fillna(0)
monthly_totals['year_month'] = monthly_totals['year_month'].astype(str)
monthly_totals["measure"] = ((monthly_totals["hawkish_total"] - monthly_totals["dovish_total"])/(monthly_totals["hawkish_total"] + monthly_totals["dovish_total"]))
monthly_totals = monthly_totals.sort_values(by='year_month')

monthly_totals = monthly_totals.drop(monthly_totals.tail(3).index)

df_PPI = pd.read_csv("PPIACO.csv")
df_PPI['DATE'] = pd.to_datetime(df_PPI['DATE'], format='%Y-%m-%d')
df_PPI["PPI_change"] =  df_PPI["PPIACO"].pct_change()*100
df_PPI = df_PPI[(df_PPI['DATE'] >= '2000-01-01') & (df_PPI['DATE'] <= '2022-06-01')]
fig = make_subplots(specs=[[{"secondary_y": True}]])

stability_multiplier = (stable_totals['stable_total']/100)
y_value = ((monthly_totals["measure"]*10)*stability_multiplier)

fig.add_trace(
    go.Scatter(x=df_PPI['DATE'], y=df_PPI["PPI_change"], name="PPI Percentage Change", marker=dict(color='red')),
    secondary_y=True,
)

# hawkish dovish measure with stability and trust multipliers
fig.add_trace(
    go.Scatter(x=monthly_totals['year_month'], y=y_value*0.95, name="Our Measure", marker=dict(color='black')),
    secondary_y=False,
)

# plain hawkish dovish measure
fig.add_trace(
    go.Scatter(x=monthly_totals['year_month'], y=monthly_totals["measure"]*10, name="Our measure w/o Multipliers"),
    secondary_y=False,
)

# hawkish dovish measure with stability
fig.add_trace(
    go.Scatter(x=monthly_totals['year_month'], y=y_value, name="Our Measure w/ Stability"),
    secondary_y=False,
)

fig.update_layout(
    title_text="Various Versions of Our Measure (2000 to 2022)",
    font=dict(
        size=18,
    )
)

fig.update_xaxes(title_text="Date")

fig.update_yaxes(title_text="Our Measure", secondary_y=False)
fig.update_yaxes(title_text="Market Volatility Index", secondary_y=True)

fig.show()

<bound method NDFrame.head of                                                 sentence        date    label  \
0      Corporations Dealer Financing Terms Senior Cre...  2021-05-26   DOVISH   
1      May 26, 2021 The Economic Outlook and Monetary...  2021-05-26   DOVISH   
2      Today, I will explain why I expect the U.S. ec...  2021-05-26   DOVISH   
3      Households and businesses adapted, supported b...  2021-05-26   DOVISH   
4      Highly accommodative monetary policy by the Fe...  2021-05-26  NEUTRAL   
...                                                  ...         ...      ...   
27301  Members also agreed that the statement should ...  2007-08-07  HAWKISH   
27302  In addition, the Committee agreed that the sta...  2007-08-07  HAWKISH   
27303  Against this backdrop, members judged that the...  2007-08-07  HAWKISH   
27304  At the conclusion of the discussion, the Commi...  2007-08-07  HAWKISH   
27305  The vote encompassed approval of the text belo...  2007-08-07  HAWKISH  

In [105]:
# Final Graph
fig = make_subplots(specs=[[{"secondary_y": True}]])

# hawkish dovish measure with stability and trust multipliers
fig.add_trace(
    go.Scatter(x=monthly_totals['year_month'], y=y_value*0.95, name="Our Measure"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_PPI['DATE'], y=df_PPI["PPI_change"], name="PPI Percentage Change"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Our Measure vs. PPI Percentage Change (2000 to 2022)",
    font=dict(
        size=18,
    )
)

fig.update_xaxes(title_text="Date")

fig.update_yaxes(title_text="Our Measure", secondary_y=False)
fig.update_yaxes(title_text="PPI Percentage Change", secondary_y=True)

fig.show()